# SciPy 2019 Tools Overview

## This notebook is an overview of tools mentioned in the SciPy 2019 Conference.
By Akira Sewnath

These tools are mostly for data science and data visualization. More information about some of these packages can be found at [www.scipy2019.scipy.org/tutorial-participant-instructions](https://www.scipy2019.scipy.org/tutorial-participant-instructions).

# SatPy

A package originally designed for quickly generating high quality, high resolution satellite imagery.

### Reading data files

SatPy centers around the class "Scene". Data is loaded into a Scene via one of the readers available.

In [ ]:
from satpy import Scene
from glob import glob

# Get the list of GOES-16 ABI files to open
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
#filenames

In [ ]:
#Check out available readers
from satpy import available_readers
sorted(available_readers())

In [ ]:
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.keys()

Files aren't actually loaded into scene until you explicitly load it.

In [ ]:
scn.available_dataset_names()

The `Scene` is telling us that we have all 16 ABI channels available to load. This list includes any product that we can load from the file that the "abi_l1b" reader is configured to access. If we didn't provide all of the necessary files or the data was missing from the file for some reason, that product would not be listed here.

| Channel     | Wavelength  |  Resolution  |
| ----------- | ----------- |  ----------- |
| C01         | 0.47µm      |  1000m       |
| C02         | 0.64µm      |  250m        |
| C03         | 0.64µm      |  1000m       |
| C04         | 1.37µm      |  2000m       |
| C05         | 1.60µm      |  1000m       |
| C06         | 2.20µm      |  2000m       |
| C07         | 3.90µm      |  2000m       |
| C08         | 6.20µm      |  2000m       |
| C09         | 6.90µm      |  2000m       |
| C10         | 7.30µm      |  2000m       |
| C11         | 8.40µm      |  2000m       |
| C12         | 9.60µm      |  2000m       |
| C13         | 10.30µm     |  2000m       |
| C14         | 11.20µm     |  2000m       |
| C15         | 12.30µm     |  2000m       |
| C16         | 13.30µm     |  2000m       |

In [ ]:
my_channel = 'C01'
scn.load([my_channel])
scn[my_channel]

<img src="../pictures/dask.png">

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
plt.figure()
plt.imshow(scn[my_channel])
plt.colorbar()

### Writing data files

In [ ]:
from satpy import available_writers
sorted(available_writers())

In [ ]:
from dask.diagnostics import ProgressBar

with ProgressBar():
  scn.save_datasets()

In [ ]:
!pwd
!ls

### Resampling

SatPy provides methods for resampling channels for comparison purposes or to change projections.

In [ ]:
scn.load(['C05'])
scn['C05'].attrs['area']

In [ ]:
scn.load(['C06'])
scn['C06'].attrs['area']

The native resampler used has two possible operations:

1. If remapping data to a higher resolution, replicate each pixel to make the shape matches.
2. If remapping data to a lower resolution, average/aggregate the pixels to make the shapes match.

By default this resamples to the highest resolution area (smallest footprint per pixel) shared between the loaded datasets. You can easily specify the lower resolution by adding the argument `scn.min_area()`.

In [ ]:
new_scn = scn.resample(resampler='native')
new_scn['C05'].shape == new_scn['C06'].shape

In [ ]:
new_scn['C06'].attrs['area']

### Composites

In [ ]:
scn.available_composite_names()

For this example, we will use the `airmass` preconfigured compsite. The airmass RGB is made of the following bands:

- R: C08 - C10
- G: C12 - C13
- B: C08

The red channel is the difference between the C08 (6.185µm) and C10 (7.34µm) bands, the green channel is the difference between the C12 (9.61µm) and C13 (10.35µm) bands, and the blue channel is the C08 (6.185µm) band.

In [ ]:
scn.load(['airmass'])
scn['airmass']

In [ ]:
from satpy.writers import get_enhanced_image

plt.figure()
img = get_enhanced_image(scn['airmass'])
img_data = img.data
img_data.plot.imshow(rgb='bands', vmin=0, vmax=1)

## Geopandas

Extends pandas to create tools specifically for geospatial data

### Reading data

Explicitly creating geodata from flat text files. Data from [insideairbnb.com](https://insideairbnb.com).

In [ ]:
import geopandas
import pandas
neighborhoods = pandas.read_csv('../data/neighborhoods.csv')
listings = pandas.read_csv('../data/listings.csv.gz')

For the first dataset, the `listings` data, records are provided with information about the latitude and longitude of the listing. You can use the latitude and longitude data to construct geometries required to create a `geodataframe`, a subclass of `pandas` dataframe useful for working with geographic data, directly from coordinates.

In [ ]:
listings.head()

In [ ]:
list(listings.columns)

The other dataset contains information about different regions in the Austin area.

In [ ]:
neighborhoods.head()

*wkb*: well known binary representation of geographical information. Often, the well-known binary representation is a string of binary digits, encoded in hexidecimal, that represents the structure of the geometry corresponding to that record in the dataframe. Neighborhoods are "areal" features, meaning that they are polygons. Thus, the well-known binary column encodes the shape of these polygons.

### Creating Geometries from Raw Coordinates

Now we will turn both datasets into *geodataframes*. For the first dataset, `geopandas` has helper functions to construct a `geodataframe`. The `geodataframe` requires a `geoseries` called `geometry`.

In [ ]:
geometries = geopandas.points_from_xy(listings.longitude, listings.latitude)

In [ ]:
geometries

In [ ]:
listings = geopandas.GeoDataFrame(listings, geometry=geometries)
listings['geometry']

Creating a `geodataframe` for the neighborhoods dataset also requires us to explicitly define a `geometry`. This is done by reading in the wkb data using `shapely`.

In [ ]:
from shapely import wkb
neighborhoods['geometry'] = neighborhoods.wkb.apply(lambda shape: wkb.loads(shape, hex=True))
neighborhoods

In [ ]:
neighborhoods.geometry[0]

In [ ]:
neighborhoods = geopandas.GeoDataFrame(neighborhoods)

Using basemap and contextily to create an image of the neighborhoods and AirBnB distributions. The coordinate reference system is set to the mercator projection for this. 

In [ ]:
#neighborhoods.crs = {'init': 'epsg:3857'}
#listings.crs = {'init':'epsg:3857'}

In [ ]:
#neighborhoods.total_bounds

In [ ]:
#import contextily
#basemap, _ = contextily.bounds2img(*neighborhoods.total_bounds, zoom=10)

In [ ]:
plt.figure(figsize=(8, 8))
neighborhoods.boundary.plot(ax=plt.gca(), color='orangered')
listings.plot(ax=plt.gca(), marker='.', markersize=5, color='green')

## Seaborn

Seaborne is a statistical visualization package in Python meant to quickly create attractive graphs.

<img src="../pictures/seaborn.png">

In [ ]:
import libpysal as lp
import numpy
import contextily
import shapely.geometry as geom
import mapclassify
%matplotlib inline

In [ ]:
df = geopandas.read_file('../data/neighborhoods.gpkg')
listings = geopandas.read_file('../data/listings.gpkg')
#preprocessing price
listings['price'] = listings.price.str.replace('$', '').str.replace(',','_').astype(float)

Code to aggregate the data within neighborhoods to look at median prices

In [ ]:
median_price = geopandas.sjoin(listings[['price', 'geometry']], df, op='within')\
                  .groupby('index_right').price.median()
df['median_pri'] = median_price.values

In [ ]:
df['median_pri'].fillna((df['median_pri'].mean()), inplace=True)
df.head()

In [ ]:
import seaborn as sbn
sbn.distplot(df['median_pri'], rug=True)

## GEODataFrame Chloropleth

`geodataframe` has a default chloropleth graph for spatially defined attributes.

In [ ]:
df.plot(column='median_pri')

## Rasterio

Package to access geospatial raster data

In [ ]:
import rasterio
nightlight_file = rasterio.open('../data/txlights.tif')
nightlight_file

Reading bands into array:

In [ ]:
nightlights = nightlight_file.read(1)
nightlights

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(nightlights, cmap='hot')

## Glue Data Visualization

Glue is a GUI meant to aid data exploration through visualization tools. We're starting by loading the Iris dataset from [archive.uci.edu](https://archive.ics.uci.edu/ml/datasets.php). The data needed a little bit of preprocessing in order to create class subsets within Glue (changed class from string to integer).

In [ ]:
data = pandas.read_csv('../data/iris.data', sep=" ", header=None)
data

Here are a couple screen shots when the Iris data is in Glue. You can select different subsets and view them in different plots to get a better understanding of your data's characteristics.

<img src="../pictures/glue_3d_scatter.png">

<img src="../pictures/glue_multiple_windows.png">